In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib
import datetime as dt

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [3]:
today_date = dt.date.today()

In [4]:
# # Create SQL Connection
# driver="ODBC Driver 17 for SQL Server"
# engine = create_engine(f'mssql://LAPTOP-E6QKON1L/client_surveillance?driver={driver}')
# engine_con = engine.connect()

In [5]:
# This creates the connection to the Bespoke Engine's Oregon Cannabis files in SQL
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=client_surveillance;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine_con = engine.connect()

### Read in and Clean

In [6]:
# Read in new data
cleaned_client_surv = pd.read_excel('Active Borrrow Worksheet/Active Borrow Worksheet 2021_04_20.xlsx', sheet_name='Cleaned Sheet', index_col=0).replace('', 'Blank')

In [7]:
cleaned_client_surv.head(15)

,Headwaters,Thirty One Labs,Dreamfields,Good Tree,SISU,Pura Cali,Buddies,iCannic,JC Rad,Kalifornia Green Akers,Leune,Space Coyote,Adventure Challenge,FlowerHire,Cannary,Herer Group,Friendly Farms,Umbrla,Luminescence Labs,Royal Apothecary,SD Strains,Floramye,Accentian,EPOD (Crown Genetics),Lobo Cannagar,DNA Organics,Voyage Distribution,HTC Solutions,HPCC,Kat's Naturals,Gold Seal,Cookies
Financing Approved:,LOC,Inventory,LOC,Inventory,Inventory,Invoice,LOC,LOC,Inventory,LOC,Inventory,LOC,Inventory,LOC,Inventory,Invoice,LOC,LOC,Inventory,Inventory,Inventory,Purchase Money Financing,Purchase Money Financing,Inventory,Inventory,LOC,Blank,Blank,Blank,Blank,Blank,Blank
Credit Limit:,3000000,250000,10000000,500000,1000000,1000000,3000000,250000,250000,250000,2000000,600000,2000000,150000,300000,100000,1200000,3000000,500000,150000,300000,338000,250000,800000,150000,200000,300000,150000,400000,200000,Blank,Blank
Effective Financing Fee:,0.0276,0.0276,0.02,0.0273,0.03,0.0297,0.0285,0.0315,0.0325,0.0325,0.0225,0.031,0.0325,0.03,0.032,0.024,0.03,0.029,0.031,0.0324,0.0305,0.0315,0.0309,0.03,0.0320001,0.03,0.03099,0.0320001,0.031,0.03201,Blank,Blank
Date of Most Recent Financials:,2021-02-28 00:00:00,2020-12-31 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,2020-06-30 00:00:00,2020-10-31 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-02-28 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-02-28 00:00:00,2021-02-28 00:00:00,2021-02-28 00:00:00,2020-11-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2020-12-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,2021-02-28 00:00:00,2021-03-31 00:00:00,2021-03-31 00:00:00,2020-11-30 00:00:00,2020-07-31 00:00:00,2021-01-31 00:00:00,2021-03-31 00:00:00,2020-09-30 00:00:00,2020-09-30 00:00:00
Working Capital Ratio,0.687897,2.04322,2.68613,0.976502,1.45236,3.7797,0.945466,0.382584,1.07184,1.14241,2.91574,3.60792,1.15234,8.57269,1.94796,1.28789,4.62527,1.4513,1.94912,0.924093,4.24737,1.33497,0.973703,2.94278,10.2101,2.271,4.32739,1.69787,1.98387,5.88182,1.34285,4.16182
Net Sales to Working Capital Ratio,-3.44819,0.756986,1.08104,-19.4176,2.01834,0.050364,-5.08216,0,16.9161,0,0.0877141,0.27433,14.9488,0.310519,0.772059,0,0.409484,0.717208,0.764551,-7.3293,0.128734,1.06152,-13.3841,0.222095,0.348595,0.838785,0.505771,0.462832,1.13288,0.252353,4.63986,1.41683
Accounts Receivable Turnover Ratio,42.4424,173.102,35.9586,12,132.884,5.26622,4.95831,12,500.04,12,3.26222,1.45871,12,6.118,13.6803,0,52.7589,15.1137,23.9226,686.041,-7.0954,25.1775,24.7222,5.15342,311.188,9.67812,6.0218,107.258,12,48.4041,7.60924,8.46295
Inventory Turnover Ratio (COGS),88.0907,0.466178,17.1317,6.20064,13.5614,0.66744,3.47726,0,41.0744,11.8623,0.779501,0,0,0,5.3904,0,3.3213,3.65682,0.0436502,13.9418,0.378009,8.68374,0,4.93604,2.87039,23.1469,3.58799,1.10782,14.9788,1.36515,27.673,4.31148
Operating Cycle (COGS),12.7434,785.072,31.456,89.2816,29.6613,616.175,178.581,30.4167,9.61626,61.1864,580.135,250.222,30.4167,59.66,94.3938,NaN,116.815,123.964,8377.2,26.7124,914.143,56.5296,14.764,144.773,128.333,53.4828,162.341,332.878,54.7844,274.91,61.1578,127.787
Interest Coverage Ratio,-1.47335,62.3639,15.0185,4.81396,30.5896,NaN,140.84,-33.1807,4.86453,-10.3663,-1.10653,NaN,NaN,19.5146,NaN,NaN,NaN,-2.61984,16.0132,0.557522,NaN,NaN,NaN,-43.0927,NaN,-13.4348,210.894,NaN,13.8041,-8.98396,4.25262,7.26915


In [8]:
# Turn it sideways
cleaned_client_surv = cleaned_client_surv.T

In [9]:
cleaned_client_surv = cleaned_client_surv.reset_index().rename(columns={'index': 'Company'})

In [10]:
# Run this is needed
cleaned_client_surv['Date of Most Recent A/R & A/P Reports:'] = pd.to_datetime(cleaned_client_surv['Date of Most Recent A/R & A/P Reports:'].replace('Blank', ''))

In [11]:
if cleaned_client_surv['Date of Most Recent A/R & A/P Reports:'].dtype == 'O' or cleaned_client_surv['Date of Most Recent Financials:'].dtype == 'O':
    print(error)

In [12]:
cleaned_client_surv = cleaned_client_surv[~cleaned_client_surv['Company'].str.contains('Unnamed')]

In [13]:
# This removes the .1 at the end of companies that appear twice, which appear because they are read in as columns
company_list = list(cleaned_client_surv['Company'])
new_company_list = []
for company in company_list:
    company = company.replace('.1', '')
    new_company_list.append(company)
cleaned_client_surv['Company'] = new_company_list

### Rename Columns

In [14]:
len(cleaned_client_surv.columns)

71

In [15]:
# This cleans out all the spaces and punctuation from the column names
old_column_names = list(cleaned_client_surv.columns)
new_columns_names = []
for n in old_column_names:
    n = n.lower()
    n = n.replace(' ', '_')
    n = n.replace('-', '_')
    n = n.replace(':', '')
    n = n.replace('/', '')
    n = n.replace('(', '')
    n = n.replace(')', '')
    n = n.replace('_&', '')
    n = n.replace('&', '')
    n = n.replace('__', '_')
    new_columns_names.append(n)
cleaned_client_surv.columns = new_columns_names

In [16]:
cleaned_client_surv.head()

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes
0,Headwaters,LOC,3000000,0.0276,2021-02-28,0.687897,-3.44819,42.4424,88.0907,12.7434,-1.47335,0,18.0542,NaN,0.00965347,0.0508064,-0.102255,-0.0160632,0.0310898,-0.131101,2021-02-28,-0.0538157,0.678858,0.175337,0.00907376,0.0228931,0.0562601,0.0562601,0.145986,0.201096,0.596613,0.306499,0.0998,0,0,0,10,10,10,0,5,10,0,5,5,1,1,5,0,0,3,3,3,5,5,5,5,5,3,5,5,5,114,Inventory Financing,LOC,Blank,Blank,Blank,Blank,Blank,Blank
1,Thirty One Labs,Inventory,250000,0.0276,2020-12-31,2.04322,0.756986,173.102,0.466178,785.072,62.3639,-0.8267,8.09863,NaN,0.957339,0.681965,0.36907,0.592122,0.183215,-0.0591972,2020-12-31,0.132496,0.340137,0,0.497704,0.267243,0.51135,0.704496,1.0561,1,0.281848,0.125334,0.26724,0.00584786,10,0,10,0,0,5,0,10,0,10,10,10,10,10,1,0,1,0,5,1,0,0,0,0,1,1,1,0,96,Inventory Financing,Inventory,Blank,Blank,Blank,Blank,Blank,Blank
2,Dreamfields,LOC,10000000,0.02,2021-03-31,2.68613,1.08104,35.9586,17.1317,31.456,15.0185,1.56199,NaN,NaN,0.339858,0.348717,0.364524,0.282069,0.274659,0.290162,2021-03-31,0.53818,0.380119,0.05958,0.0186495,0.0846995,0.151252,0.192882,0.233931,0.264137,1,0,0,0,10,0,10,10,10,5,1,10,0,10,10,10,10,10,10,1,5,5,5,5,3,5,5,5,5,5,5,5,175,Line of Credit,LOC,Blank,Blank,Blank,Blank,Blank,Blank
3,Good Tree,Inventory,500000,0.0273,2021-02-28,0.976502,-19.4176,12,6.20064,89.2816,4.81396,31.9439,NaN,NaN,0.380562,0.324972,0.319826,0.139371,0.0418165,0.043034,2021-02-28,1,0,0,0,1,1,1,1,1,0,0.0518583,0.0152013,0,0,0,5,1,5,0,0,10,0,10,10,10,10,5,5,5,5,5,5,0,0,0,0,0,0,0,0,0,91,Inventory Financing,Inventory,Blank,Blank,Blank,Blank,Blank,Blank
4,SISU,Inventory,1000000,0.03,2021-01-31,1.45236,2.01834,132.884,13.5614,29.6613,30.5896,6.44511,NaN,NaN,0.1866,0.12967,0.126472,0.102202,0.0579148,0.0477631,2021-01-31,0.762635,0.194652,0.110116,0.0120847,0.2

In [17]:
# This  column will help us keep track of companies that we are actively working with
cleaned_client_surv['active'] = 1

In [18]:
cleaned_client_surv['date_added'] = today_date

### Read in the SQL Data to Compare

In [19]:
old_sql = pd.read_sql('client_surveillance_all', con=engine_con)

In [20]:
old_sql.head()

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,#_of_quarters_of_runway_remaining_payable_performance,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added
0,Headwaters,LOC,3000000,0.0276,2020-12-31,1.54989,2.31484,29.4046,53.6853,19.2119,-57.1177,0,1.32805,None,-0.457051,-0.115967,-0.0667266,-0.50706,-0.152772,-0.0958241,2020-12-31,0.0572086,0.701247,0.0623293,0.103968,0.195926,0.381126,0.460615,0.532727,0.59686,0.860997,0.139003,0,0,None,10,3,10,10,10,0,5,0,0,0,1,1,0,0,1,0,3,3,5,3,0,0,0,1,5,5,5,5,86,Fails Underwriting,LOC,None,None,None,None,None,None,1,2021-03-30
1,Thirty One Labs,Inventory,250000,0.0276,2020-11-30,2.45435,0.1618,52.2319,4.06704,96.7339,-13.913,-0.848649,3.41476,None,0.0255356,0.033968,0.085443,-0.642766,-0.358994,-0.379897,2020-11-30,0,0.221712,0.7365,0.00485844,0.376483,0.720373,0.912724,0.977437,1.01654,0.0599092,0.547119,0.103157,0.00350898,None,10,0,10,1,1,0,0,1,0,5,5,5,0,0,0,0,0,5,5,0,0,0,0,0,0,3,3,1,55,Purchase Money Financing,Inventory,None,None,None,None,None,None,1,2021-03-30
2,Dreamfields,LOC,3000000,0.0285,2020-11-30,2.35985,0.803772,28.4829,7.41881,62.014,None,1.83132,None,None,0.47087,0.470104,0.405343,0.394463,0.393393,0.326811,2020-11-30,0.408129,0.497379,0,0,0.0635764,0.124124,0.177148,0.210941,0.239096,0.96063,0.0229906,-0.000526656,0,None,10,0,10,1,5,5,1,10,0,10,10,10,10,10,10,1,5,5,5,5,5,5,5,5,5,5,5,5,163,Line of Credit,LOC,None,None,None,None,None,None,1,2021-03-30
3,Good Tree,Inventory,500000,0.0273,2020-12-31,0.442909,-0.254959,12,5.10482,101.918,1.17885,0,-9.20697,None,0.307246,0.300559,0.298905,0.0460108,-0.0725238,-0.178474,2020-12-31,1,0,0,0,1,1,1,1,1,0.0150684,0.0246448,0,0.894403,None,0,0,5,1,1,0,5,0,0,10,10,10,5,1,0,5,5,5,5,0,0,0,0,0,0,0,0,5,73,Fails Underwriting,Inventory,None,None,None,None,None,None,1,2021-03-30
4,SISU,Inventory,1000000,0.03,2020-12-31,2.11258,1.01511,110.13,12.6545,3

In [21]:
old_sql[(old_sql['company'] == 'HPCC') & (old_sql['date_of_most_recent_financials'] == '2021-01-31') & (old_sql['date_of_most_recent_ar_ap_reports'] == '2021-01-31')]

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,#_of_quarters_of_runway_remaining_payable_performance,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added
44,HPCC,,,,2021-01-31,1.98387,1.13288,12,14.9788,54.7844,13.8041,0.418988,None,None,0.406999,0.332749,0.321833,0.137309,0.0179848,0.0238148,2021-01-31,1,0,0,0,1,1,1,1,1,0.452695,0.314572,0.157843,0.0515738,None,10,0,5,10,10,5,5,10,0,10,10,10,10,5,5,5,5,5,5,0,0,0,0,0,1,3,5,5,139,Line of Credit,0,None,None,None,None,None,None,1,2021-03-30


In [22]:
already_here = []
for n in range(len(old_sql)):
    name = old_sql['company'][n]
    date_fin = old_sql['date_of_most_recent_financials'][n]
    date_rep = old_sql['date_of_most_recent_ar_ap_reports'][n]
    name_and_date = f'{name} {date_fin} {date_rep}'
    already_here.append(name_and_date)

In [23]:
new_sql = pd.DataFrame(columns=list(old_sql.columns))

In [24]:
for n in range(len(cleaned_client_surv)):
    name = cleaned_client_surv['company'][n]
    date_fin = cleaned_client_surv['date_of_most_recent_financials'][n]
    date_rep = cleaned_client_surv['date_of_most_recent_ar_ap_reports'][n]
    name_and_date = f'{name} {date_fin} {date_rep}'
    if name_and_date not in already_here:
        print(name_and_date)
        new_sql = new_sql.append(cleaned_client_surv.iloc[n])

In [25]:
new_sql.head()

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,#_of_quarters_of_runway_remaining_payable_performance,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added


In [26]:
fix_list = str('company	working_capital_score	net_sales_to_working_capital_score	accounts_receivable_turnover_score	inventory_turnover_score	operating_cycle_days	interest_coverage_score	debt_to_equity_score	number_of_quarters_of_runway_remaining_new	personal_guarantee_support_liquid_assets__proposed_credit_limit	gross_margin_score_past_quarter	gross_margin_score_past_2_quarters	gross_margin_score_past_3_quarters	net_income_margin_score_past_quarter	net_income_margin_score_past_2_quarters	net_income_margin_score_past_3_quarters	number_of_ar_current	number_of_ar_1_30_days_delinquent	number_of_ar_31_60_days_delinquent	number_of_ar_61_90_days_delinquent	largest_payee_exposure_score	total_top_2_payee_exposure_score	total_top_3_payee_exposure_score	total_top_4_payee_exposure_score	total_top_5_payee_exposure_score	number_of_ap_current	number_of_ap_1_30_days_delinquent	number_of_ap_31_60_days_delinquent	number_of_ap_61_90_days_delinquent	score_total	potential_financing_eligibility	currently_approved_financing	quarterly_net_revenue	ar_balance	ar_as_percent_of_net_revenue	ar_as_percent_of_net_revenue_score	ar_diversification_score	accrues_federal_income_taxes')

In [27]:
fix_list.replace('\t', ', ')

'company, working_capital_score, net_sales_to_working_capital_score, accounts_receivable_turnover_score, inventory_turnover_score, operating_cycle_days, interest_coverage_score, debt_to_equity_score, number_of_quarters_of_runway_remaining_new, personal_guarantee_support_liquid_assets__proposed_credit_limit, gross_margin_score_past_quarter, gross_margin_score_past_2_quarters, gross_margin_score_past_3_quarters, net_income_margin_score_past_quarter, net_income_margin_score_past_2_quarters, net_income_margin_score_past_3_quarters, number_of_ar_current, number_of_ar_1_30_days_delinquent, number_of_ar_31_60_days_delinquent, number_of_ar_61_90_days_delinquent, largest_payee_exposure_score, total_top_2_payee_exposure_score, total_top_3_payee_exposure_score, total_top_4_payee_exposure_score, total_top_5_payee_exposure_score, number_of_ap_current, number_of_ap_1_30_days_delinquent, number_of_ap_31_60_days_delinquent, number_of_ap_61_90_days_delinquent, score_total, potential_financing_eligibili

### Main Alter

In [28]:
# This will be the dataframe that we use to make the changes
main_alter_df = pd.DataFrame()

In [29]:
# This should narror down to 
for comp in old_sql['company'].unique():
    
    old_surv = pd.DataFrame(data=old_sql[old_sql['company'] == comp])
    old_surv = old_surv.reset_index().drop(columns={'index'})
    
    new_surv = pd.DataFrame(data=cleaned_client_surv[cleaned_client_surv['company'] == comp])
    new_surv = new_surv[new_surv['date_of_most_recent_financials'] == old_surv['date_of_most_recent_financials'][0]]
    new_surv = new_surv[new_surv['date_of_most_recent_ar_ap_reports'] == old_surv['date_of_most_recent_ar_ap_reports'][0]]
    new_surv = new_surv.reset_index().drop(columns={'index'})
    
#     if old_surv == new_surv):
#         print(comp)
#     else:
#         main_alter_df.append(new_surv)

In [30]:
old_surv

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,#_of_quarters_of_runway_remaining_payable_performance,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added
0,Gold Seal,Blank,Blank,Blank,2020-09-30,1.34285,4.63986,7.60924,27.673,61.1578,4.25262,3.54775,None,None,0.11068,0.00369834,0.0433668,0.0634615,-0.0397553,0.000319038,2020-12-18,0.129622,0.540755,0.0833626,0.0384784,0.00514805,0.00995301,0.0117253,0.013593,0.015784,0.547522,0.255988,0.0426191,0.122233,Blank,5,5,1,10,5,0,0,10,0,5,5,5,5,1,5,0,3,3,1,5,5,5,5,5,1,5,3,5,108,Inventory Financing,0,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-03-31


In [31]:
new_surv

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added
0,Gold Seal,Blank,Blank,Blank,2020-09-30,1.34285,4.63986,7.60924,27.673,61.1578,4.25262,3.54775,NaN,NaN,0.11068,0.00369834,0.0433668,0.0634615,-0.0397553,0.000319038,2020-12-18,0.129622,0.540755,0.0833626,0.0384784,0.00514805,0.00995301,0.0117253,0.013593,0.015784,0.547522,0.255988,0.0426191,0.122233,5,5,1,10,5,0,0,10,0,5,5,5,5,1,5,0,3,3,1,5,5,5,5,5,1,5,3,5,108,Inventory Financing,0,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-05-05


In [32]:
cleaned_client_surv.head()

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added
0,Headwaters,LOC,3000000,0.0276,2021-02-28,0.687897,-3.44819,42.4424,88.0907,12.7434,-1.47335,0,18.0542,NaN,0.00965347,0.0508064,-0.102255,-0.0160632,0.0310898,-0.131101,2021-02-28,-0.0538157,0.678858,0.175337,0.00907376,0.0228931,0.0562601,0.0562601,0.145986,0.201096,0.596613,0.306499,0.0998,0,0,0,10,10,10,0,5,10,0,5,5,1,1,5,0,0,3,3,3,5,5,5,5,5,3,5,5,5,114,Inventory Financing,LOC,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-05-05
1,Thirty One Labs,Inventory,250000,0.0276,2020-12-31,2.04322,0.756986,173.102,0.466178,785.072,62.3639,-0.8267,8.09863,NaN,0.957339,0.681965,0.36907,0.592122,0.183215,-0.0591972,2020-12-31,0.132496,0.340137,0,0.497704,0.267243,0.51135,0.704496,1.0561,1,0.281848,0.125334,0.26724,0.00584786,10,0,10,0,0,5,0,10,0,10,10,10,10,10,1,0,1,0,5,1,0,0,0,0,1,1,1,0,96,Inventory Financing,Inventory,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-05-05
2,Dreamfields,LOC,10000000,0.02,2021-03-31,2.68613,1.08104,35.9586,17.1317,31.456,15.0185,1.56199,NaN,NaN,0.339858,0.348717,0.364524,0.282069,0.274659,0.290162,2021-03-31,0.53818,0.380119,0.05958,0.0186495,0.0846995,0.151252,0.192882,0.233931,0.264137,1,0,0,0,10,0,10,10,10,5,1,10,0,10,10,10,10,10,10,1,5,5,5,5,3,5,5,5,5,5,5,5,175,Line of Credit,LOC,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-05-05
3,Good Tree,Inventory,500000,0.0273,2021-02-28,0.976502,-19.4176,12,6.20064,89.2816,4.81396,31.9439,NaN,NaN,0.380562,0.324972,0.319826,0.139371,0.0418165,0.043034,2021-02-28,1,0,0,0,1,1,1,1,1,0,0.0518583,0.0152013,0,0,0,5,1,5,0,0,10,0,10,10,10,10,5,5,5,5,5,5,0,0,0,0,0,0,0,0,0,91,Inventory Financing,Inventory,Blank,Blank,Blank,Blank,Blank,Blank,1,2021-05-05
4,SISU,Inventory,1000000,0.03,2021-01-31,1.45236,2.01834,132.884,13.5614,29.6613,30.5896,6.44511,NaN,NaN,0.1866,0.12967,0.126472,0.102202,0

In [33]:
new_sql

,company,financing_approved,credit_limit,effective_financing_fee,date_of_most_recent_financials,working_capital_ratio,net_sales_to_working_capital_ratio,accounts_receivable_turnover_ratio,inventory_turnover_ratio_cogs,operating_cycle_cogs,interest_coverage_ratio,debt_to_equity_ratio,#_of_quarters_of_runway_remaining_financial_ratios,personal_guarantee_support_assets_proposed_credit_limit,gross_margin_ratio_past_quarter,gross_margin_ratio_past_2_quarters,gross_margin_ratio_past_3_quarters,net_income_margin_ratio_past_quarter,net_income_margin_ratio_past_2_quarters,net_income_margin_ratio_past_3_quarters,date_of_most_recent_ar_ap_reports,percent_of_ar_current,percent_of_ar_1_30_days_delinquent,percent_of_ar_31_60_days_delinquent,percent_of_ar_61_90_days_delinquent,largest_percent_payee_exposure,total_top_2_payee_exposure_percent,total_top_3_payee_exposure_percent,total_top_4_payee_exposure_percent,total_top_5_payee_exposure_percent,percent_of_ap_current,percent_of_ap_1_30_days_delinquent,percent_of_ap_31_60_days_delinquent,percent_of_ap_61_90_days_delinquent,#_of_quarters_of_runway_remaining_payable_performance,working_capital_score,net_sales_to_working_capital_score,accounts_receivable_turnover_score,inventory_turnover_score,operating_cycle_days,interest_coverage_score,debt_to_equity_score,number_of_quarters_of_runway_remaining_new,personal_guarantee_support_liquid_assets_proposed_credit_limit,gross_margin_score_past_quarter,gross_margin_score_past_2_quarters,gross_margin_score_past_3_quarters,net_income_margin_score_past_quarter,net_income_margin_score_past_2_quarters,net_income_margin_score_past_3_quarters,number_of_ar_current,number_of_ar_1_30_days_delinquent,number_of_ar_31_60_days_delinquent,number_of_ar_61_90_days_delinquent,largest_payee_exposure_score,total_top_2_payee_exposure_score,total_top_3_payee_exposure_score,total_top_4_payee_exposure_score,total_top_5_payee_exposure_score,number_of_ap_current,number_of_ap_1_30_days_delinquent,number_of_ap_31_60_days_delinquent,number_of_ap_61_90_days_delinquent,score_total,potential_financing_eligibility,currently_approved_financing,quarterly_net_revenue,ar_balance,ar_as_percent_of_net_revenue,ar_as_percent_of_net_revenue_score,ar_diversification_score,accrues_federal_income_taxes,active,date_added


In [34]:
stop

NameError: name 'stop' is not defined

In [ ]:
new_sql.to_sql('client_surveillance_all', con=engine_con, index=False, if_exists='append')

In [ ]:
# test = pd.read_sql('client_surveillance_all', con=engine_con)
# test = test.replace('Blank', '')
# test.to_sql('client_surveillance_all', con=engine_con, index=False, if_exists='replace')